In [1]:
# coding: utf-8

import os
import sys
import pickle
import time
import datetime
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import useful_methods


# *******************************************************
# *******************************************************

# Limitations
TIME_LIMIT = 108
RETWEET_STATUS = True
FILTER_STATUS = True
START_TIME = 1
END_TIME = 108


# *******************************************************
# *******************************************************

# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')
dfGameInfos = useful_methods.DropNanGames(dfGameInfos)

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

In [2]:
all_tweet = []
all_home = []
all_away = []
filter_home = []
filter_away = []
non_rt_home = []
non_rt_away = []

for ith_row in range(len(dfGameInfos)):
    team_home = dfGameInfos.iloc[ith_row]['home_team']
    team_away = dfGameInfos.iloc[ith_row]['away_team']
    week = dfGameInfos.iloc[ith_row]['GW']
    
    # read df
    df = useful_methods.SingleGameDf(week, team_home, team_away, filtering=False, retweet=True)
    df.ith_minute = [int(item) for item in df.ith_minute]
    df = df[(df.ith_minute >= START_TIME) & (df.ith_minute <= END_TIME)].copy().reset_index(drop=True)

    # append tweet nums
    all_tweet.append(len(df))

    all_home.append(len(df[df.side == 'home']))
    all_away.append(len(df[df.side == 'away']))
    
    filter_home.append(len(df[
                (df.side == 'home') &
                (df.user_status == 'normal') &
                (df.text_status == 'normal')]))
    filter_away.append(len(df[
                (df.side == 'away') &
                (df.user_status == 'normal') &
                (df.text_status == 'normal')]))
    
    non_rt_home.append(len(df[
                (df.side == 'home') &
                (df.user_status == 'normal') &
                (df.text_status == 'normal') &
                (df.status != 'retweet')
            ]))
    
    non_rt_away.append(len(df[
                (df.side == 'away') &
                (df.user_status == 'normal') &
                (df.text_status == 'normal') &
                (df.status != 'retweet')
            ]))
    
    print(week, team_home, team_away)

5 Everton Chelsea
5 Norwich Bournemouth
5 Watford Swansea
5 WestBromwich Southampton
5 Arsenal Stoke
5 Crystal City
5 United Liverpool
5 Sunderland Tottenham
5 Leicester Villa
5 WestHam Newcastle
6 Chelsea Arsenal
6 Swansea Everton
6 Bournemouth Sunderland
6 Villa WestBromwich
6 Newcastle Watford
6 Stoke Leicester
6 City WestHam
6 Tottenham Crystal
6 Southampton United
6 Liverpool Norwich
7 Tottenham City
7 Leicester Arsenal
7 Stoke Bournemouth
7 WestHam Norwich
7 Southampton Swansea
7 United Sunderland
7 Liverpool Villa
7 Newcastle Chelsea
7 WestBromwich Everton
8 Crystal WestBromwich
8 Norwich Leicester
8 Sunderland WestHam
8 City Newcastle
8 Villa Stoke
8 Bournemouth Watford
8 Chelsea Southampton
8 Everton Liverpool
8 Swansea Tottenham
8 Arsenal United
9 Tottenham Liverpool
9 Southampton Leicester
9 WestBromwich Sunderland
9 Chelsea Villa
9 Crystal WestHam
9 Everton United
9 City Bournemouth
9 Watford Arsenal
9 Newcastle Norwich
9 Swansea Stoke
10 Norwich WestBromwich
10 Leicester C

In [7]:
# add counted nums
dfGameInfos['all_tweet'] = all_tweet
dfGameInfos['all_home'] = all_home
dfGameInfos['all_away'] = all_away
dfGameInfos['filter_home'] = filter_home
dfGameInfos['filter_away'] = filter_away
dfGameInfos['non_rt_home'] = non_rt_home
dfGameInfos['non_rt_away'] = non_rt_away

# Drop columns
dfTweetNum = dfGameInfos[['GW', 'home_team', 'away_team'] + list(dfGameInfos.columns[-7:])].copy()
dfTweetNum

,GW,home_team,away_team,all_tweet,all_home,all_away,filter_home,filter_away,non_rt_home,non_rt_away
0,5,Everton,Chelsea,23899,3029,11802,2985,11784,2149,7276
1,5,Norwich,Bournemouth,1158,523,212,522,212,334,149
2,5,Watford,Swansea,856,231,207,227,204,145,101
3,5,WestBromwich,Southampton,1091,489,309,313,305,182,184
4,5,Arsenal,Stoke,9077,6636,358,5826,355,3691,233
5,5,Crystal,City,5745,536,3089,534,3058,399,1185
6,5,United,Liverpool,47218,25902,10848,25734,10700,10640,6526
7,5,Sunderland,Tottenham,4299,1047,2814,1039,2785,657,1595
8,5,Leicester,Villa,7053,526,2682,520,2669,399,1840
9,5,WestHam,Newcastle,6819,1781,3420,1458,3383,820,2350


In [8]:
# Save as CSV
useful_methods.DFtoCSV(dfTweetNum, paths.DATA_HOME + "EPL/", 'tweet_numbers', index=False)
print("[Saved in]: %s" % (paths.DATA_HOME + "EPL/" + 'tweet_numbers.csv'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/tweet_numbers.csv


In [16]:
dic_tw_nums = {}

teams = list(set(dfGameInfos.home_team))
teams.sort()


for team in teams:
    df = dfGameInfos[(dfGameInfos.home_team == team) | (dfGameInfos.away_team == team)].copy().reset_index(drop=True)

    dic_tw_nums[team] = []

    for ith_row in range(len(df)):
        team_home = df.iloc[ith_row]['home_team']
        team_away = df.iloc[ith_row]['away_team']

        if team == team_home:
            dic_tw_nums[team].append(df.iloc[ith_row]['non_rt_home'])
        else:
            dic_tw_nums[team].append(df.iloc[ith_row]['non_rt_away'])

In [26]:

for key in list(dic_tw_nums.keys()).sort():
    print(key, np.array(dic_tw_nums[key]).mean())

TypeError: 'NoneType' object is not iterable